<a href="https://colab.research.google.com/github/jhonnye0/edge-impulse-project/blob/main/GlucoseWatch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [2]:
df = pd.read_csv("diabetes.csv")

# Funções

In [3]:
def class_to_number(x):
    if x > 200:
        return 2
    if x > 120 and x <= 200:
        return 1
    return 0

def number_to_class(x):
    if x[2] == 1:
        return '200-400'
    if x[1] == 1:
        return '120-200'
    return '70-120'

def number_to_class2(x: int):
    if x > 200:
        return 2
    if x > 120 and x <= 200:
        return 1
    return 0

In [4]:
def normalize_hour(df):
    df['Horário'] = pd.to_datetime(df['Timestamp'])
    hour = df['Horário'].dt.hour
    minute = df['Horário'].dt.minute
    second = df['Horário'].dt.second
    seconds_after_midnight = hour * 3600 + minute * 60 + second
    df['Horário'] = seconds_after_midnight / (24*60*60)
    df['Horário'] = df['Horário'].astype('float64')

# Pré-processamento

In [5]:
df

,Glicose,IMC,Temperatura,Pressão,Oxigenação,BPM,TempAmbiente,Timestamp,Diabetes
0,79,26.3,35.2,77/124,97,74,27.0,2024-03-14 20:00:00,0
1,90,26.4,36.0,77/126,97,69,30.0,2024-03-14 11:00:00,0
2,83,26.4,35.2,75/118,97,75,32.0,2024-03-14 13:00:00,0
3,131,26.4,36.3,77/120,99,87,32.0,2024-03-14 14:00:00,0
4,101,26.4,36.2,80/128,98,73,31.0,2024-03-14 15:00:00,0
5,97,26.4,36.3,79/125,97,73,28.0,2024-03-14 18:00:00,0
6,105,26.5,35.6,74/110,98,61,27.0,2024-03-14 19:00:00,0
7,111,21.3,36.3,73/123,97,58,26.0,2024-03-15 07:00:00,1
8,143,21.3,36.3,75/105,97,64,28.0,2024-03-15 07:30:00,1
9,75,21.3,36.4,79/128,95,75,16.0,2024-03-15 12:00:00,1


In [6]:
df["Pressão"] = df["Pressão"].apply(lambda x: eval(x))

In [7]:
df['Horário'] = pd.to_datetime(df['Timestamp'])

In [8]:
df.describe()

,Glicose,IMC,Temperatura,Pressão,Oxigenação,BPM,TempAmbiente,Diabetes
count,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000
mean,128.157895,23.736842,36.181579,0.643230,96.236842,69.315789,26.736842,0.526316
std,43.833547,2.561083,0.381926,0.029191,2.398583,8.094393,4.618905,0.506009
min,75.000000,21.300000,35.200000,0.593496,88.000000,58.000000,16.000000,0.000000
25%,94.750000,21.300000,36.000000,0.625000,95.250000,62.250000,27.000000,0.000000
50%,119.000000,21.450000,36.300000,0.636730,97.000000,69.500000,27.000000,1.000000
75%,143.750000,26.400000,36.400000,0.656607,97.750000,75.000000,30.000000,1.000000
max,245.000000,26.500000,36.800000,0.714286,99.000000,87.000000,32.000000,1.000000


In [9]:
column_names_to_normalize = ['IMC', 'Temperatura', 'Pressão', 'Oxigenação', 'BPM', 'TempAmbiente']
x = df[column_names_to_normalize].values
mm_scaler = preprocessing.MinMaxScaler()
x_scaled = mm_scaler.fit_transform(x)
df[column_names_to_normalize] = x_scaled
normalize_hour(df)

In [10]:
df.dtypes

Glicose           int64
IMC             float64
Temperatura     float64
Pressão         float64
Oxigenação      float64
BPM             float64
TempAmbiente    float64
Timestamp        object
Diabetes          int64
Horário         float64
dtype: object

In [11]:
oldGlic = df['Glicose'].copy()

In [12]:
df['Glicose'] = df['Glicose'].apply(number_to_class2)

In [13]:
df.sort_values('Timestamp', ignore_index=True, inplace=True)
time_c = df.pop('Timestamp')
df['Timestamp'] = time_c

In [14]:
df.to_csv("diabetes2.csv", sep=',', encoding='utf-8', index=False)

In [15]:
df['Glicose'] = oldGlic

# Modelo

In [16]:
# Criação do modelo
model = Sequential()

# Adicionando a camada de entrada e a primeira camada oculta
model.add(Dense(units=32, activation='relu', input_dim=8))

# Adicionando a segunda camada oculta
model.add(Dense(units=16, activation='relu'))

# Adicionando a camada de saída
model.add(Dense(units=3, activation='softmax'))

# Compilando o modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Exibindo o resumo do modelo
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 3)                 51        
                                                                 
Total params: 867 (3.39 KB)
Trainable params: 867 (3.39 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Treinamento

In [17]:
X = df.drop(['Glicose', 'Timestamp'], axis=1).values
y = to_categorical(df['Glicose'].apply(class_to_number).values)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [19]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

In [20]:
# Treinando o modelo
model.fit(X_train, y_train, epochs=100, batch_size=8, validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/100
4/4 [==============================] - 3s 251ms/step - loss: 1.0453 - accuracy: 0.5000 - val_loss: 0.9416 - val_accuracy: 0.7000
Epoch 2/100
4/4 [==============================] - 0s 27ms/step - loss: 1.0265 - accuracy: 0.5000 - val_loss: 0.9116 - val_accuracy: 0.7000
Epoch 3/100
4/4 [==============================] - 0s 30ms/step - loss: 1.0149 - accuracy: 0.5357 - val_loss: 0.8834 - val_accuracy: 0.7000
Epoch 4/100
4/4 [==============================] - 0s 24ms/step - loss: 1.0038 - accuracy: 0.5357 - val_loss: 0.8597 - val_accuracy: 0.6000
Epoch 5/100
4/4 [==============================] - 0s 45ms/step - loss: 0.9988 - accuracy: 0.5714 - val_loss: 0.8365 - val_accuracy: 0.7000
Epoch 6/100
4/4 [==============================] - 0s 26ms/step - loss: 0.9911 - accuracy: 0.5357 - val_loss: 0.8227 - val_accuracy: 0.7000
Epoch 7/100
4/4 [==============================] - 0s 33ms/step - loss: 0.9855 - accuracy: 0.5357 - val_loss: 0.8087 - val_accuracy: 0.7000
Epoch 8/100
4/4 [==

# Avaliação do modelo

In [21]:
loss = model.evaluate(X_test, y_test)

print('Loss nos dados de teste:', loss)

1/1 [==============================] - 0s 103ms/step - loss: 0.7581 - accuracy: 0.8000
Loss nos dados de teste: [0.7581046223640442, 0.800000011920929]


In [22]:
y_pred = model.predict(X_test)

1/1 [==============================] - 0s 317ms/step


In [23]:
y_pred_classes = to_categorical(np.argmax(y_pred, axis=1), num_classes=3)
for r,p in zip(y_test, y_pred_classes):
    print("Real: ", number_to_class(r), "\nPrev: ", number_to_class(p))
    print()

Real:  70-120 
Prev:  70-120

Real:  70-120 
Prev:  70-120

Real:  70-120 
Prev:  70-120

Real:  120-200 
Prev:  120-200

Real:  70-120 
Prev:  70-120

Real:  70-120 
Prev:  120-200

Real:  70-120 
Prev:  70-120

Real:  70-120 
Prev:  120-200

Real:  70-120 
Prev:  70-120

Real:  120-200 
Prev:  120-200



In [24]:
acuracia = accuracy_score(y_test, y_pred_classes)
print('Acurácia do modelo:', acuracia)

Acurácia do modelo: 0.8


# TFLite

In [25]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)